In [1]:
using MultiPoly
using Interact

In [2]:
function copy_dict_values(dict::Dict)
    Dict(zip(keys(dict), deepcopy(values(dict))))
end

copy_dict_values (generic function with 1 method)

In [3]:
import SoftRobots
reload("SoftRobots")

In [ ]:
world = SoftRobots.World3D()
world_state = Dict{SoftRobots.Object, SoftRobots.ObjectState}()

robot, robot_state = SoftRobots.blob(num_nodes=50, k=100)
for i = 1:length(robot_state.positions)
    robot_state.positions[i] += SoftRobots.Point(0., 0, 1.)
end
push!(world.objects, robot)
world_state[robot] = robot_state

x, y, z = generators(MPoly{Float64}, :x, :y, :z)
terrain = SoftRobots.FixedObject()
terrain_state = SoftRobots.FixedObjectState(0 + (0.0x^2  + 0.0y^2 + 1z))
push!(world.objects, terrain)
world_state[terrain] = terrain_state

history = typeof(world_state)[]

# Profile.clear()
SoftRobots.update!(world, world_state, 0.001)
@time for j = 1:5000
    SoftRobots.update!(world, world_state, 0.001)
    push!(history, copy_dict_values(world_state))
    SoftRobots.draw(robot, world_state[robot])
end

# playback(history, 10)
@manipulate for i = 1:length(history)
    SoftRobots.draw(robot, history[i][robot])
end